;# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [112]:
print("Welcome to the Challenge Trading Planner")

Welcome to the Challenge Trading Planner


In [113]:
print("We are first going to plan the month using a compounding calculator")
# ---- Step 1: Get user input with defaults ----
starting_capital_default = 100
rate_default = 5  # in percent
periods_default = 30

starting_capital = float(input(f"Enter Starting Capital [{starting_capital_default}]: ") or starting_capital_default)
rate = float(input(f"Enter Period Gain Rate (%) [{rate_default}]: ") or rate_default)
periods = int(input(f"Enter number of periods [{periods_default}]: ") or periods_default)

print(f"Starting Capital: {starting_capital}")
print(f"Period count: {period_count}")
print(f"Period gain rate: {per_period_gain_rate}")



We are first going to plan the month using a compounding calculator
Starting Capital: 10.0
Period count: 30
Period gain rate: 5


In [114]:
import pandas as pd

# ---- Step 2: Calculate compounded values ----
data = []
ultimate_gain = 0
for period in range(1, periods + 1):
    value = principal * ((1 + rate / 100) ** period)
    ultimate_gain = value
    data.append({"Period": period, "Value": round(value, 2)})


# ---- Step 3: Create DataFrame and print ----
df = pd.DataFrame(data)

# Show all rows
pd.set_option('display.max_rows', None)

# Show all columns
pd.set_option('display.max_columns', None)

# Optional: prevent column width truncation
pd.set_option('display.max_colwidth', None)

# print
df


,Period,Value
0,1,12.00
1,2,14.40
2,3,17.28
3,4,20.74
4,5,24.88
5,6,29.86
6,7,35.83
7,8,43.00
8,9,51.60
9,10,61.92


In [115]:
ultimate_gain = round(ultimate_gain, 2)
ultimate_growth = round(ultimate_gain /starting_capital * 100, 2)
print(f"Ultimate Gain Target: {ultimate_gain} currency -> {ultimate_growth} %")

Ultimate Gain Target: 2373.76 currency -> 23737.6 %


In [119]:
print("--- Risk parameters ---")
rr = 4
max_margin_account_rate = 10 # max % of account put into a position at any given time
max_account_rate_at_risk = 1 # max % of account losable per trade
print(f"RR: {rr}")
print(f"Max Margin per Trade: {max_margin_account_rate} % of account")
print(f"Max Losable: {max_account_rate_at_risk}% of account")
print("")

# calculate RR
# calculate % at risk
# calculate what SL/Exit should be with the risk params.
# Calculate what the position size should be for the risk param and entries.

leverage = 300 # in x
position_value = 100 # in currency
rounding = 6
margin = round(position_value / leverage, rounding)
margin_v_account = round(margin / starting_capital * 100, rounding)
print("--- Position Parameters ---")
print(f"leverage: {leverage}x")
print(f"Position Value: {position_value} currency")
margin_grade = "✅" if margin_v_account <= max_margin_account_rate else "❌"
print(f"Margin: {margin} currency -> {margin_v_account}% of Account {margin_grade}")
print("")

long_entry = 162 # in currency
long_exit = 163 # in currency
long_sl = 159 # in currency
long_position_size = round(position_value / long_entry, rounding) # number of asset in the trade

long_at_risk = round(long_position_size * (long_entry - long_sl) / leverage, rounding)
long_to_win = round(long_position_size * (long_exit - long_entry) / leverage, rounding)
long_win_pa_percentage = round((long_exit - long_entry) / long_entry * 100, 3)
long_loss_pa_percentage = round((long_sl - long_entry) / long_entry * 100, 3)
long_rr = round(long_to_win/long_at_risk, 2)

long_at_risk_v_account = round(long_at_risk/starting_capital *100, rounding)
long_to_win_v_account = round(long_to_win/starting_capital *100, rounding)

print("--- Long ---")
print(f"Entry: {long_entry}")
print(f"Exit: {long_exit} -> +{long_win_pa_percentage}% PA")
print(f"SL: {long_sl} -> {long_loss_pa_percentage}% PA")
long_rr_grade_pass = True if long_rr >= rr else False
long_rr_grade = "✅" if long_rr_grade_pass else "❌"
print(f"RR: {long_rr} {long_rr_grade}")
if long_rr_grade_pass is False:
    long_new_exit = 165#FIXME: here
    print(f"==> Change exit: {long_new_exit}")
    print("== OR ==")
    long_new_sl = 161.5
    print(f"==> Change SL:{long_new_sl}")
print("")

print("---")
print(f"Capital in position: {margin} currency")
print(f"Position size: {long_position_size} assets")
long_at_risk_grade = "✅" if long_at_risk_v_account <= max_account_rate_at_risk else "❌"
print(f"At risk: {long_at_risk} currency -> {long_at_risk_v_account}% of account {long_at_risk_grade}")
print(f"To win: {long_to_win} currency -> {long_to_win_v_account}% of account")


short_entry = 100 # in currency
short_exit = 70 # in currency
short_sl = 110 # in currency

--- Risk parameters ---
RR: 4
Max Margin per Trade: 10 % of account
Max Losable: 1% of account

--- Position Parameters ---
leverage: 300x
Position Value: 100 currency
Margin: 0.333333 currency -> 3.33333% of Account ✅

--- Long ---
Entry: 162
Exit: 163 -> +0.617% PA
SL: 159 -> -1.852% PA
RR: 0.33 ❌
==> Change exit: 165
== OR ==
==> Change SL:161.5

---
Capital in position: 0.333333 currency
Position size: 0.617284 assets
At risk: 0.006173 currency -> 0.06173% of account ✅
To win: 0.002058 currency -> 0.02058% of account


In [122]:
if long_rr_grade_pass is False:
    print("--- Found some misalignement. ---")
    print("=> Let's adjust for your compounding goals and risk parameters")


    if long_rr_grade_pass is False:
        print(f"RR goals (1:{rr}) is not met, here are suggestions")
        long_new_exit = (long_entry - long_sl) * rr + long_entry
        print(f"1/ Adjust your TP to {long_new_exit} (from {long_exit}) -> {long_new_exit - long_exit} delta")
        long_new_sl = long_entry - (long_entry - long_entry) / (rr-1)
        print(f"2/ Adjust your SL to {long_new_sl} (from {long_sl}) -> {long_new_sl - long_sl} delta")


--- Found some misalignement. ---
=> Let's adjust for your compounding goals and risk parameters
RR goals (1:4) is not met, here are suggestions
1/ Adjust your TP to 174 (from 163) -> 11 delta
2/ Adjust your SL to 162.0 (from 159) -> 3.0 delta
